In [1]:
import pandas as pd
import glob

# big_smiles 폴더 안의 polyBERT_len85_로 시작하는 모든 csv 경로 가져오기
paths = sorted(glob.glob("big_smiles/polyBERT_len85_*.csv"))

# 각 파일을 읽어서 리스트로 만든 다음
dfs = [pd.read_csv(p) for p in paths]

# 하나의 DataFrame으로 합치기
df = pd.concat(dfs, ignore_index=True)

print(len(paths), "files loaded")
print(df.shape)


50 files loaded
(4926212, 2)


In [2]:
big_smiles=df['0'].tolist()

In [ ]:
# bigsmiles_chemberta_mlm.py

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

# 1) BigSMILES 리스트 (실제에선 외부에서 읽어오면 됨)
# big_smiles = [...]


# 2) HF Dataset으로 감싸기
dataset = Dataset.from_dict({"text": big_smiles})

# 3) ChemBERTa 토크나이저 & 모델 불러오기
base_model_name = "DeepChem/ChemBERTa-77M-MTR"  # 예시, HF에 있는 ChemBERTa 중 하나
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# (선택) BigSMILES 특수 기호 추가
#  - ChemBERTa vocab에 없을 가능성이 높은 것들: "{", "}", "$", "[*]" 등
new_tokens = []
for tok in ["{", "}", "$", "[*]"]:
    if tok not in tokenizer.vocab:
        new_tokens.append(tok)

if new_tokens:
    num_added = tokenizer.add_tokens(new_tokens)
    print(f"Added {num_added} new tokens: {new_tokens}")
else:
    num_added = 0
    print("No new tokens added.")

model = AutoModelForMaskedLM.from_pretrained(base_model_name)

if num_added > 0:
    model.resize_token_embeddings(len(tokenizer))

# 4) 토크나이징 함수
def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )

tokenized_dataset = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"],
)

# 5) MLM용 collator (BERT-style 15% 마스킹 자동 처리)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,
)

# 6) TrainingArguments & Trainer

training_args = TrainingArguments(
    output_dir="bigsmiles-chemberta-mlm",
    per_device_train_batch_size=32,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,

    # ↓↓↓ 여기 2줄 추가
    logging_dir="runs/bigsmiles_mlm",   # TensorBoard 로그 저장 폴더
    report_to=["tensorboard"],          # wandb 말고 텐서보드로 보고
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# 7) 학습
trainer.train()

# 8) 저장
trainer.save_model("bigsmiles-mlm")
tokenizer.save_pretrained("bigsmiles-mlm")
print("Saved to bigsmiles-mlm/")
